<a href="https://colab.research.google.com/github/vitor-thompson/Python-para-financas-investimento-e-analise-de-dados./blob/main/Modelo_de_precifica%C3%A7%C3%A3o_de_ativos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Modelo de precificação de ativos.

In [1]:
# Instala a biblioteca yfinance, usada para baixar dados financeiros.
!pip install -q yfinance

# Importa as bibliotecas necessárias para manipulação de dados numéricos (numpy) e DataFrames (pandas),
# além do yfinance para download de dados de mercado.
import numpy as np
import pandas as pd
import yfinance as yf

# Define os tickers (símbolos) das ações que queremos analisar e o índice de mercado (^BVSP para Ibovespa).
tikers = ['ITUB4.SA','ITSA4.SA','TAEE11.SA','SBSP3.SA','VALE3.SA','PETR4.SA', 'RADL3.SA','WEGE3.SA','BBSE3.SA','RAIL3.SA', '^BVSP']

# Cria um DataFrame vazio que será preenchido com os preços de fechamento das ações e do índice.
data = pd.DataFrame()

# Loop para baixar os preços de fechamento históricos para cada ticker.
# Os dados são baixados para o período de 2012-01-01 a 2016-01-01.
# A coluna 'Close' (fechamento) é selecionada e adicionada ao DataFrame 'data'.
for t in tikers:
    data[t] = yf.download(t, start='2012-1-1', end = '2016-1-1')['Close']

/tmp/ipython-input-4215144966.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data[t] = yf.download(t, start='2012-1-1', end = '2016-1-1')['Close']
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-4215144966.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data[t] = yf.download(t, start='2012-1-1', end = '2016-1-1')['Close']
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-4215144966.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data[t] = yf.download(t, start='2012-1-1', end = '2016-1-1')['Close']
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-4215144966.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data[t] = yf.download(t, start='2012-1-1', end = '2016-1-1')['Close']
[*********************100%*******************

In [2]:
# Calcula os retornos logarítmicos diários para as ações e o índice de mercado.
# Os retornos logarítmicos (ou contínuos) são preferidos em análises financeiras
# pois permitem que os retornos sejam aditivos ao longo do tempo e seguem uma distribuição normal.
# A fórmula é log(Pt / Pt-1), onde Pt é o preço atual e Pt-1 é o preço anterior.
sec_returns = np.log(data / data.shift(1))

In [ ]:
# Define o número de dias úteis em um ano.
# Este valor é usado para anualizar (escalar para um ano) as métricas de risco, como covariância e variância.
dias_uteis = 250

# Calcula a matriz de covariância anualizada dos retornos logarítmicos.
# A covariância mede como os retornos de dois ativos se movem em conjunto (se sobem e descem juntos).
# Multiplicar por 'dias_uteis' anualiza a covariância diária.
cov = sec_returns.cov() * dias_uteis

# Exibe a matriz de covariância anualizada.
cov

In [7]:
# Extrai da matriz de covariância (cov) as covariâncias de cada ação com o Ibovespa ('^BVSP').
# 'tikers[:-1]' seleciona todos os tickers exceto o último (que é o '^BVSP'),
# e '^BVSP' seleciona a coluna correspondente ao Ibovespa.
cov_stocks_with_ibovespa = cov.loc[tikers[:-1], '^BVSP']

# Exibe as covariâncias das ações com o Ibovespa.
display(cov_stocks_with_ibovespa)

,^BVSP
ITUB4.SA,0.052782
ITSA4.SA,0.051435
TAEE11.SA,0.012084
SBSP3.SA,0.038854
VALE3.SA,0.051760
PETR4.SA,0.078481
RADL3.SA,0.021112
WEGE3.SA,0.012505
BBSE3.SA,0.041272
RAIL3.SA,0.057009


In [13]:
# Itera sobre cada ação e sua respectiva covariância com o Ibovespa.
# Imprime o valor da covariância formatado para 6 casas decimais e o tipo de dado para verificação.
for stock, covariance_value in cov_stocks_with_ibovespa.items():
    print(f"A covariância de {stock} com o Ibovespa é: {covariance_value:.6f} (Tipo: {type(covariance_value)})")

A covariância de ITUB4.SA com o Ibovespa é: 0.052782 (Tipo: <class 'float'>)
A covariância de ITSA4.SA com o Ibovespa é: 0.051435 (Tipo: <class 'float'>)
A covariância de TAEE11.SA com o Ibovespa é: 0.012084 (Tipo: <class 'float'>)
A covariância de SBSP3.SA com o Ibovespa é: 0.038854 (Tipo: <class 'float'>)
A covariância de VALE3.SA com o Ibovespa é: 0.051760 (Tipo: <class 'float'>)
A covariância de PETR4.SA com o Ibovespa é: 0.078481 (Tipo: <class 'float'>)
A covariância de RADL3.SA com o Ibovespa é: 0.021112 (Tipo: <class 'float'>)
A covariância de WEGE3.SA com o Ibovespa é: 0.012505 (Tipo: <class 'float'>)
A covariância de BBSE3.SA com o Ibovespa é: 0.041272 (Tipo: <class 'float'>)
A covariância de RAIL3.SA com o Ibovespa é: 0.057009 (Tipo: <class 'float'>)


In [14]:
# Calcula a variância anualizada dos retornos do mercado (Ibovespa, '^BVSP').
# A variância do mercado é um componente crucial para o cálculo do Beta, pois mede a volatilidade do mercado.
# Multiplicar por 'dias_uteis' anualiza a variância diária.
market_var = sec_returns['^BVSP'].var() * dias_uteis

# Exibe a variância anualizada do mercado.
market_var

0.0508467896591147

In [15]:
# Calcula o Beta para cada ação.
# O Beta é uma medida de risco sistemático, indicando a sensibilidade do retorno de uma ação
# em relação ao retorno do mercado (Ibovespa).
# É calculado como a covariância da ação com o mercado dividida pela variância do mercado.
beta_stocks = cov_stocks_with_ibovespa / market_var

# Exibe os valores de Beta para cada ação.
display(beta_stocks)

,^BVSP
ITUB4.SA,1.038065
ITSA4.SA,1.011562
TAEE11.SA,0.237650
SBSP3.SA,0.764148
VALE3.SA,1.017961
PETR4.SA,1.543478
RADL3.SA,0.415211
WEGE3.SA,0.245942
BBSE3.SA,0.811687
RAIL3.SA,1.121188


In [17]:
# Define a taxa livre de risco (risk-free rate) e o prêmio de risco do mercado.
# A taxa livre de risco é o retorno de um investimento sem risco (como um título do governo).
# O prêmio de risco de mercado é o retorno adicional que os investidores esperam por assumir o risco do mercado.
taxa_livre_de_risco_BR = 0.1378 # Taxa livre de risco no Brasil (exemplo: Selic)
premio_de_risco = 0.05       # Prêmio de risco esperado do mercado

# Calcula o retorno esperado de cada ação usando o CAPM (Capital Asset Pricing Model).
# O CAPM é um modelo que estima o retorno esperado de um ativo, considerando seu risco sistemático (Beta).
# A fórmula é: Retorno Esperado = Taxa Livre de Risco + Beta * Prêmio de Risco.
expected_returns = taxa_livre_de_risco_BR + beta_stocks * premio_de_risco

# Exibe os retornos esperados calculados para cada ação.
display(expected_returns)

,^BVSP
ITUB4.SA,0.189703
ITSA4.SA,0.188378
TAEE11.SA,0.149682
SBSP3.SA,0.176007
VALE3.SA,0.188698
PETR4.SA,0.214974
RADL3.SA,0.158561
WEGE3.SA,0.150097
BBSE3.SA,0.178384
RAIL3.SA,0.193859


In [19]:
# Calcula o Índice de Sharpe para cada ação.
# O Índice de Sharpe mede o retorno excedente (acima da taxa livre de risco) por unidade de risco (volatilidade).
# sharp = (Retorno Esperado - Taxa Livre de Risco) / Volatilidade do Ativo.

# Calcula o desvio padrão (volatilidade) anualizado de cada ação.
# É necessário excluir o '^BVSP' pois o cálculo do Sharpe é para ações individuais.
volatilities = sec_returns[tikers[:-1]].std() * (dias_uteis ** 0.5)

# Calcula o Sharpe Ratio para cada ação.
# Usa expected_returns (retorno esperado para cada ação) e taxa_livre_de_risco_BR.
sharp_ratios = (expected_returns - taxa_livre_de_risco_BR) / volatilities

# Exibe os Índices de Sharpe calculados.
display(sharp_ratios)

,0
ITUB4.SA,0.169164
ITSA4.SA,0.170850
TAEE11.SA,0.044589
SBSP3.SA,0.107720
VALE3.SA,0.130717
PETR4.SA,0.164777
RADL3.SA,0.068008
WEGE3.SA,0.050434
BBSE3.SA,0.129621
RAIL3.SA,0.091099
